In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates

import scipy.signal              #importing scipy.signal package for detrending
from scipy.fftpack import fft    #importing Fourier transform package
from scipy.stats import chi2     #importing confidence interval package
from scipy.stats import norm

from datetime import datetime

In [ ]:
# data = pd.read_csv("208681_20230918_1216_data.txt",sep = ",",skiprows = 0, parse_dates = ['Time'])
def get_pressure_data(ps_number, create_plots=False):
    """
    This function is used to get pressure data. 
    The resulting dataset uses time [datetime], pressure [Pa], sea pressure [Pa], depth [m]
    
    ps_number: number of pressure sensor to test
    
    create_plots (boolean): if True, create plots of the data
    """
    
    for filename in os.listdir(f'ps_data/raw_data/ps{ps_number}'):
        if 'data.txt' in filename:
            fn = filename
            
    data = pd.read_csv(f'ps_data/raw_data/ps{ps_number}/{fn}', parse_dates = ['Time'])
    
    data['Pressure'] = data['Pressure'] * 10000
    
    data['Sea pressure'] = data['Sea pressure'] * 10000
    
    time = data[['Time']].to_numpy()
    pressure = data[['Pressure']].to_numpy()
    sea_pressure = data[['Sea pressure']].to_numpy()
    depth = data[['Depth']].to_numpy()
    
    if create_plots:
        fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
        plt.subplots_adjust(left=None, bottom= 2, right=None, top=3, wspace=None, hspace=None)


        ax[0].plot(time, pressure)
        ax[0].set_title("Pressure")

        ax[1].plot(time, sea_pressure)
        ax[1].set_title("Sea_Pressure")

        ax[2].plot(time, depth)
        ax[2].set_title("Depth")
        
    return data

In [ ]:
data_ps2_rec1 = get_pressure_data(2, create_plots=True)

In [ ]:
display(data_ps2_rec1)

In [ ]:
def zoom_to(data, t_begin, t_end, create_plots=True):
    """
    Returns dataframe of data (Time, Pressure, Sea_Pressure, Depth) between a given begin and end time.
    
    data (dataframe)
    t_begin (string, see format): begin time
    t_end (string, see format): end time
    create_plots: if True, function creates plots
    
    DateTime format: [YYYY-MM-DD hh:mm:ss] (or Datetime object)
    """
    
    if type(t_begin) != 'datetime.datetime':
        t_begin = datetime.strptime(t_begin, '%Y-%m-%d %H:%M:%S')
    if type(t_end) != 'datetime.datetime':
        t_end = datetime.strptime(t_end, '%Y-%m-%d %H:%M:%S')
        
    ds = data[data['Time'] > t_begin]
    ds = ds[ds['Time'] < t_end]
        
    if create_plots == True:
        time = ds['Time']
        pressure = ds['Pressure']
        sea_pressure = ds['Sea pressure']
        depth = ds['Depth']
        
        fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
        plt.subplots_adjust(left=None, bottom= 2, right=None, top=3, wspace=None, hspace=None)

        ax[0].plot(time, pressure)
        ax[0].set_title("Pressure")

        ax[1].plot(time, sea_pressure)
        ax[1].set_title("Sea_Pressure")

        ax[2].plot(time, depth)
        ax[2].set_title("Depth")
        
    return ds

test_ds = zoom_to(data_ps2_rec1, '2023-09-23 19:00:00', '2023-09-23 20:00:00');

In [ ]:
def wave_spectrum(data, nfft, Fs):
    ''' Compute variance density spectrum from a given time-series and its 
    90% confidence intervals. 
    The time-series is first divided into blocks of length nfft before being 
    Fourier-transformed 
    (Note that this is one of the simplest ways to estimate the variance density spectrum 
    (no overlap between the blocks, and a rectangular window) - see for instance 
    scipy.signal.welch for more advanced calculations)

    INPUT
      data    timeseries 
      nfft    block length
       Fs     sampling frequency of the timeseries (Hz)
    
    OUTPUT
      E       (one-sided) variance spectral density. If data is in meters, E is in m^2/Hz
      f       frequency axis (Hz)
      confLow and confUpper     Lower and upper 90% confidence interval; 
                                (Multiplication factors for E)  '''
    
    # 1. PRELIMINARY CALCULATIONS
    # ---------------------------
    n = len(data)                # length of the time-series
    nfft = int(nfft - (nfft%2))  # the length of the window should be an even number

    data = scipy.signal.detrend(data)      # detrend the time-series
    nBlocks = int(n/nfft)        # number of blocks (use of int to make it an integer)

    data_new = data[0:nBlocks*nfft] # (we work only with the blocks which are complete)

    # we organize the initial time-series into blocks of length nfft 
    dataBlock = np.reshape(data_new,(nBlocks,nfft))  # each column of dataBlock is one block
    
    # 2. CALCULATION VARIANCE DENSITY SPECTRUM
    # ----------------------------------------

    # definition frequency axis
    df = Fs/nfft      # frequency resolution of the spectrum df = 1/[Duration of one block]
    f = np.arange(0,Fs/2+df,df)   # frequency axis (Fs/2 = Fnyquist = max frequency)
    fId = np.arange(0,len(f))

    # Calculate the variance for each block and for each frequency
    fft_data = fft(dataBlock,n = nfft,axis = 1)/nfft    # Fourier transform of the data
    fft_data = 2*fft_data[:,fId]                        # Only one side needed
     
    E = np.abs(fft_data)**2/2                  # E(i,b) = ai^2/2 = variance at frequency fi for block b
    phi = np.angle(fft_data)
    
    # We finally average the variance over the blocks, and divide by df to get the variance DENSITY spectrum
    E = np.mean(E, axis = 0)/df
    phi = np.mean(phi, axis=0)
    
    # 3. CONFIDENCE INTERVALS
    # -----------------------
    edf = round(nBlocks*2)   # Degrees of freedom 
    alpha = 0.1              # calculation of the 90% confidence interval

    confLow = edf/chi2.ppf(1-alpha/2,edf)    # see explanations on confidence intervals given in lecture 3 
    confUpper  = edf/chi2.ppf(alpha/2,edf)
    
    return E,phi,f,confLow,confUpper

In [ ]:
def frequency_filter(E, phi, f, f_bands):
    """
    This function takes an E, f and phi, and removes the values within the specified f_bands.
    
    E (array): energy density
    phi (array): absangles
    f (array): frequency
    f_bands (list of tuples): list of frequency bands to remove from the energy density spectrum, phase spectrum, 
                              and frequency array
                              
    returns filtered E, phi, f
    """
    
    for fmin, fmax in f_bands:
        
        
        idx = (((f < fmin) + (f > fmax)) == 0)
        
        E[idx] = 0
        phi[idx] = 0
        
    return E, phi, f

In [ ]:
E_test, phi_test, f_test, CL, CU = wave_spectrum(data=test_ds['Pressure'], nfft=2000, Fs=1/0.125)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].plot(f_test, E_test)
ax[0].set_xscale('log')

ax[1].plot(f_test, phi_test)
ax[1].set_xscale('log')

E_test1, phi_test1, f_test1 = frequency_filter(E_test, phi_test, f_test, f_bands=[(0.1, 0.5)])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].plot(f_test1, E_test1)
ax[0].set_xscale('log')

ax[1].plot(f_test1, phi_test1)
ax[1].set_xscale('log')

In [ ]:
def remove_atmospheric_pressure(data, use_default_atm_data=True, atm_data=None, create_plots=False):
    """
    Removes interpolated atmospheric pressure data from Hoek van Holland from the data
    
    data (pd.DataFrame): data to remove atmospheric pressure from
    use_default_atm_data (boolean): use prepared atmospheric pressure data 
                                    (interpolated from https://www.daggegevens.knmi.nl/klimatologie/uurgegevens)
    atm_data (pd.DataFrame): specify if use_default_atm_data False
    """
    
    if use_default_atm_data:
        atm_ps_interpolated = pd.read_csv('atm_ps_data/atm_ps_data_interpolated.txt', parse_dates=['Time'])
        
        t_begin = data['Time'][0]
        t_end = data['Time'][-1]
        
        atm_ps_interpolated = zoom_to(atm_ps_data, t_begin, t_end, create_plots=False)
        
        data['atm_removed'] = data['Pressure'] - atm_ps_interpolated['P']
        
        if create_plots:
            
            fig, axs = plt.subplots(2, 1, figsize=(15, 6))
            
            ax[0].plot(data['Time'], data['Pressure'])
            ax[0].set_title('Pressure')
            
            ax[1].plot(data['Time'], data['atm_removed'])
            ax[1].set_title('atm_removed')
            
        return data
        
    else:
        return

In [ ]:
def remove_dry_data(data):
    pass


In [ ]:
test_ds = zoom_to(data_ps2_rec1, '2023-09-23 18:26:48', '2023-09-23 18:26:50');
test_ds = zoom_to(data_ps2_rec1, '2023-09-23 18:20:00', '2023-09-23 18:30:00');

def remove_spikes():
    pass